# DeepAR / Probalistic RNN

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    # Convert 'date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    #df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [3]:
# Load in the train and test data
train_df = load_and_prepare_data('../../data/Final_data/train_df.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df.csv')

# Concatenate the train and test data
df = pd.concat([train_df, test_df])
df['Date'] = pd.to_datetime(df['Date'])

# Show 
df

,Date,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
755,2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
756,2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000
757,2024-07-27,80.74,176.67,2.05,19.63,110.479,33.307,74.958,184.012,216.412,50.927,18.856,244.051,2.180,32.001,-251.655,992,74.79,32.90,0.000


In [4]:
# Define the future covariates columns from your dataframe
future_covariates_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 
                             'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 
                             'Natural_gas (GWh)', 'Other (GWh)', 'Pumped_storage_generation (GWh)', 
                             'Solar_energy (GWh)', 'Wind_offshore (GWh)', 'Wind_onshore (GWh)', 
                             'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)', 
                             'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']

In [5]:
import torch
torch.cuda.empty_cache()  # Add this before fitting the model to clear GPU memory


: 

In [6]:
import traceback  
import pandas as pd
from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, mae, rmse, mse
from sklearn.preprocessing import MaxAbsScaler
import optuna
import plotly.graph_objects as go
from darts.utils.callbacks import TFMProgressBar
from darts.utils.likelihood_models import GaussianLikelihood

# Create time series objects
series_train = TimeSeries.from_dataframe(train_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')
series_test = TimeSeries.from_dataframe(test_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')

# Convert future covariates to TimeSeries objects

future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')

# Determine required start date for future covariates during prediction
max_input_chunk_length = 200  # Maximum input_chunk_length from your hyperparameter search
required_covariate_start = series_test.start_time() - pd.DateOffset(days=(max_input_chunk_length - 1))
required_covariate_end = series_test.end_time()

# Ensure future_covariates_full covers the required range
future_covariates_full = TimeSeries.from_dataframe(
    df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D"
).astype('float32')

# Slice future covariates for prediction
future_covariates_for_prediction = future_covariates_full.slice(
    required_covariate_start, required_covariate_end
)

# Scaling the data
scaler_series = Scaler(MaxAbsScaler())
scaler_covariates = Scaler(MaxAbsScaler())

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_for_prediction_scaled = scaler_covariates.transform(future_covariates_for_prediction)

# Define the Optuna objective function without backtesting
def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_int('hidden_dim', 10, 100)
    n_layers = trial.suggest_int('n_layers', 1, 5)
    dropout = trial.suggest_uniform('dropout', 0.0, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    input_chunk_length = trial.suggest_int('input_chunk_length', 30, 100)
    n_epochs = 5 # Adjust epochs for optimization

    # Ensure training_length >= input_chunk_length
    training_length = max(200, input_chunk_length)

    # Define the model
    model = RNNModel(
        model='LSTM',
        input_chunk_length=input_chunk_length,
        training_length=training_length,
        hidden_dim=hidden_dim,
        n_rnn_layers=n_layers,
        dropout=dropout,
        likelihood=GaussianLikelihood(),
        batch_size=batch_size,
        n_epochs=n_epochs,
        optimizer_kwargs={'lr': learning_rate},
        random_state=42,
        save_checkpoints=True,
        model_name= "rnn_model",
        pl_trainer_kwargs={
            'accelerator': 'gpu',  # Use GPU if available
            'devices': 1,
            'enable_progress_bar': True,
            'logger': False,
            'enable_model_summary': False,
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    )

    try:
        # Fit the model on the training data
        model.fit(
            series_train_scaled,
            future_covariates=future_covariates_train_scaled,
            verbose=False
        )

        # Determine forecast horizon
        n = len(series_test_scaled)

        # Make predictions
        forecast_val = model.predict(
            n=n,
            future_covariates=future_covariates_for_prediction_scaled
        )

        # Calculate MAPE on the validation set
        error = mape(series_test_scaled, forecast_val)
    except Exception as e:
        print(f'Exception during model training: {e}')
        traceback.print_exc()
        return float('inf')

    return error

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Get the best hyperparameters
best_params = study.best_params
print('Best hyperparameters:')
for key, value in best_params.items():
    print(f'  {key}: {value}')

# Ensure training_length >= input_chunk_length
best_training_length = max(200, best_params['input_chunk_length'])

# Train the best model on the full training data
best_model = RNNModel(
    model='LSTM',
    input_chunk_length=best_params['input_chunk_length'],
    training_length=best_training_length,
    hidden_dim=best_params['hidden_dim'],
    n_rnn_layers=best_params['n_layers'],
    dropout=best_params['dropout'],
    likelihood=GaussianLikelihood(),
    batch_size=best_params['batch_size'],
    n_epochs=10, 
    optimizer_kwargs={'lr': best_params['learning_rate']},
    random_state=42,
    pl_trainer_kwargs={
        'accelerator': 'gpu',  
        'devices': 1,
        'enable_progress_bar': True,
        'logger': False,
        'enable_model_summary': False,
    }
)

best_model.fit(
    series_train_scaled,
    future_covariates=future_covariates_train_scaled,
    verbose=True
)

# Determine forecast horizon based on available covariates
n = len(series_test_scaled)

# Make predictions on the test set
forecast = best_model.predict(
    n=n,
    future_covariates=future_covariates_for_prediction_scaled
)

# Inverse transform the forecast and test_series
forecast = scaler_series.inverse_transform(forecast)
test_series = scaler_series.inverse_transform(series_test_scaled)

# Plot the forecast
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=test_series.time_index,
    y=test_series.values().squeeze(),
    mode='lines',
    name='Actual',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=forecast.time_index,
    y=forecast.values().squeeze(),
    mode='lines',
    name='Forecast',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='RNN Model - Time Series Forecast',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top',
        bordercolor='black',
        borderwidth=1
    ),
    template='plotly_white'
)

fig.show()

# Calculate and print error metrics
print('Error Metrics on Test Set:')
print(f'  Mean Absolute Percentage Error (MAPE): {mape(test_series, forecast):.2f}%')
print(f'  Mean Absolute Error (MAE): {mae(test_series, forecast):.2f}')
print(f'  Root Mean Squared Error (RMSE): {rmse(test_series, forecast):.2f}')
print(f'  Mean Squared Error (MSE): {mse(test_series, forecast):.2f}')


[I 2024-09-27 09:06:38,246] A new study created in memory with name: no-name-9621a773-0324-44b9-a6f8-213777818090
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Training: 0it [00:00, ?it/s]

In [1]:
# save results and the plot 
# Save the created figure as png file and the error metrics 
fig.write_image("../../predictions/Deep_AR/_epochs_30.png")
error_metrics = pd.DataFrame({'MAE': [mae(series_test, forecast)], 'MAPE': [mape(series_test, forecast)], 'MSE': [mse(series_test, forecast)], 'RMSE': [rmse(series_test, forecast)]})
error_metrics.to_csv('../../predictions/Deep_AR/epochs_30.csv', index=False)

NameError: name 'fig' is not defined